In [128]:
import ctypes

def theSame(f1, f2, epsilon):
     return abs(f1 - f2) < epsilon
    
def theSameTuple(f1, f2, epsilon):
    return theSame(f1[0], f2[0], epsilon) and theSame(f1[1], f2[1], epsilon) and theSame(f1[2], f2[2], epsilon) \
        and theSame(f1[3], f2[3], epsilon) and theSame(f1[4], f2[4], epsilon) and theSame(f1[5], f2[5], epsilon)

def calculate_utilizations(hash_size=64, epsilon=0.001, mod_precision=100000, iterations = 2888):   
    epsilon_mod = epsilon * mod_precision
    i = 0
    
    hash_table = [(0,0,0,0,0,0)] * hash_size
    utilization = [0] * hash_size
    stored = 0
    new_stored = 0
    found = 0

    with open("input/input_acc.csv") as f1:
        with open("input/input_gyro.csv") as f2:
            for line1, line2 in zip(f1, f2):
                
                
                if i > 0:
                    # unsigned int ax_ = ((unsigned int)((unsigned int)(accX * MOD_PRECISION)) / EPSILON) % HASHSIZE;
                    #acc_x_, acc_y_, acc_z_, gyro_x_, gyro_y_, gyro_z_ = float(acc_x), float(acc_y), float(acc_z), float(gyro_x), float(gyro_y), float(gyro_z)

                    acc_x, acc_y, acc_z = [float(coordinate.strip()) for coordinate in line1.split(",")[1:4]]
                    gyro_x, gyro_y, gyro_z = [float(coordinate.strip()) for coordinate in line1.split(",")[1:4]]
                    
                    acc_x_hash = int(int(acc_x * mod_precision) / epsilon_mod)

                    hash_ = acc_x_hash % hash_size

                    utilization[hash_] += 1

                    if hash_table[hash_] == (0,0,0,0,0,0):
                        hash_table[hash_] = (acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z)
                        new_stored += 1
                    elif theSameTuple((acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z), hash_table[hash_], epsilon):
                        found += 1
                    else:
                        hash_table[hash_] = (acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z)
                        stored += 1
                    
                i += 1
                if (i > iterations): break
    
    return (utilization, new_stored, stored, found, epsilon_mod)

In [129]:
(utilization, new_stored, stored, found, epsilon_mod) = calculate_utilizations(hash_size=8, epsilon=0.5, mod_precision=10, iterations = 1000)

In [105]:
print("No. of times found: " + str(found))
print("No. of times stored: " + str(new_stored + stored) + " (New: {0}, Overwritten: {1})".format(new_stored, stored))

No. of times found: 771
No. of times stored: 229 (New: 3, Overwritten: 226)


In [106]:
sum(utilization)

1000

In [107]:
max(utilization) - min(utilization)

929

In [108]:
max(utilization)

929

In [109]:
hash_sizes = [8, 16, 32, 64, 100, 120]
epsilon=[0.00001, 0.0001, 0.001, 0.01, 0.1, 0.5]
mod_precision=[10,100,1000,10000,100000]
iterations = 1000

min_balance = 9999
max_found = 0
max_utilized = 0

selected_hash_size = 8
selected_epsilon = 0.1
selected_mod_precision = 10
selected_utilization = []

balance_obtained = 0

def utilized(utilization):
    utilized = 0
    for i in range(len(utilization)):
        if (utilization[i] > 0):
            utilized += 1
            
    return utilized

for h in hash_sizes:
    for e in epsilon:
        for m in mod_precision:
            (utilization, new_stored, stored, found, epsilon_mod) = calculate_utilizations(hash_size=h, epsilon=e, mod_precision=m, iterations = iterations)
            balance = max(utilization) - min(utilization)
            utilized_ = utilized(utilization)
            #if utilized_ > max_utilized:
            if found > max_found: # and balance < min_balance:
            #if balance < min_balance:
                selected_utilization = utilization
                selected_hash_size = h
                selected_epsilon = e
                selected_mod_precision = m
                selected_epsilon_mod = epsilon_mod
                balance_obtained = balance
                
                ns = new_stored
                s = stored
                f = found
                
                max_found = found
                min_balance = balance
                max_utilized = utilized_ 

print("Hash size: {0}, Epsilon: {1}, Mod precision: {2}, Epsilon mod: {3}".format(selected_hash_size, selected_epsilon, selected_mod_precision, selected_epsilon_mod))
print("Utilization: " + str(selected_utilization))
print("Balance obtained: {0}".format(balance_obtained))

print("Other details: ")
print("- Written: {0} (Stored fresh: {1}, Overwritten: {2})".format(ns + s, ns, s))
print("- Found: {0}".format(f))

Hash size: 8, Epsilon: 0.5, Mod precision: 10, Epsilon mod: 5.0
Utilization: [929, 63, 8, 0, 0, 0, 0, 0]
Balance obtained: 929
Other details: 
- Written: 229 (Stored fresh: 3, Overwritten: 226)
- Found: 771


In [110]:
import os
from experiment.helpers import *

verbose=2

props_and_values = {
    "HASHSIZE": selected_hash_size,
    "ITERATIONS": iterations,
    "EPSILON" : '{0:f}'.format(selected_epsilon),
    "EPSILON_MOD": int(selected_epsilon_mod),
    "MOD_PRECISION" : selected_mod_precision
}

x86_kalman_dir = os.path.join(os.path.dirname(os.getcwd()),"KalmanC++")
x86_kalman_hash_header = os.path.join(x86_kalman_dir, "HashTable", "hashtable.h")

hashtable_header_file_path = x86_kalman_hash_header
hashtable_header_file_clone_path = os.path.join(x86_kalman_dir, "Originals", "hashtable.h")
replace_define_value_in_file(hashtable_header_file_clone_path, hashtable_header_file_path, props_and_values)

if verbose >= 2: print("File '" + hashtable_header_file_path + "' now contains:")
if verbose >= 2: print_file(hashtable_header_file_path, "#define HASHSIZE")
if verbose >= 2: print_file(hashtable_header_file_path, "#define EPSILON")
if verbose >= 2: print_file(hashtable_header_file_path, "#define EPSILON_MOD")
if verbose >= 2: print_file(hashtable_header_file_path, "#define MOD_PRECISION")
if verbose >= 2: print_file(hashtable_header_file_path, "#define ITERATIONS")

File 'C:\Users\Dragos\Dropbox\RISE\Git\KalmanC++\HashTable\hashtable.h' now contains:
#define HASHSIZE 8
#define EPSILON 0.500000
#define EPSILON_MOD 5
#define MOD_PRECISION 10
#define ITERATIONS 1000


In [111]:
compile_hash()
run_hash(verbose=verbose)

Epsilon: 0.500000
Epsilon mod: 5
Mod precision: 10
Hash size: 8

HashTable population: 4 / 8
No. of times found: 245
No. of times stored: 755


**Trying a comparison function without fabs**

In [ ]:
def theSame2(f1, f2, mod_precision, epsilon, printing=False):
    epsilon_mod = int(epsilon * mod_precision)
    
    f1_mod = int(f1 * mod_precision)
    f2_mod = int(f2 * mod_precision)
    
    if printing: print("(({1} - {0}) > -{2}) and ({0} - {1} < {2})".format(f1_mod, f2_mod, epsilon_mod))
    if printing: print("(({0}) > -{2}) and ({1} < {2})".format(f2_mod - f1_mod, f1_mod - f2_mod, epsilon_mod))
    return ((f2_mod - f1_mod) > -epsilon_mod) and \
        ((f2_mod - f1_mod) < epsilon_mod) and \
        ((f1_mod - f2_mod) > -epsilon_mod) and \
        ((f1_mod - f2_mod) < epsilon_mod)

In [255]:
theSame(-10, -9.9999, 10, 0.1, printing=True)

((-99 - -100) > -1) and (-100 - -99 < 1)
((1) > -1) and (-1 < 1)


False

In [280]:
theSame2(-10, -9.9999, 10, 0.1)

False

In [254]:
import numpy as np

for a in np.arange(-10, 10, 0.0001):
    for b in np.arange(-10, 10, 0.0001):
        if (theSame(a, b, 10, 0.1) != theSame2(a, b, 10, 0.1)):
            print("{0}, {1}", a, b)

{0}, {1} -10.0 -9.9999
{0}, {1} -10.0 -9.9998
{0}, {1} -10.0 -9.9997
{0}, {1} -10.0 -9.999600000000001
{0}, {1} -10.0 -9.999500000000001
{0}, {1} -10.0 -9.999400000000001
{0}, {1} -10.0 -9.999300000000002
{0}, {1} -10.0 -9.999200000000002
{0}, {1} -10.0 -9.999100000000002
{0}, {1} -10.0 -9.999000000000002
{0}, {1} -10.0 -9.998900000000003
{0}, {1} -10.0 -9.998800000000003
{0}, {1} -10.0 -9.998700000000003
{0}, {1} -10.0 -9.998600000000003
{0}, {1} -10.0 -9.998500000000003
{0}, {1} -10.0 -9.998400000000004
{0}, {1} -10.0 -9.998300000000004
{0}, {1} -10.0 -9.998200000000004
{0}, {1} -10.0 -9.998100000000004
{0}, {1} -10.0 -9.998000000000005
{0}, {1} -10.0 -9.997900000000005
{0}, {1} -10.0 -9.997800000000005
{0}, {1} -10.0 -9.997700000000005
{0}, {1} -10.0 -9.997600000000006
{0}, {1} -10.0 -9.997500000000006
{0}, {1} -10.0 -9.997400000000006
{0}, {1} -10.0 -9.997300000000006
{0}, {1} -10.0 -9.997200000000007
{0}, {1} -10.0 -9.997100000000007
{0}, {1} -10.0 -9.997000000000007
{0}, {1} -10.

KeyboardInterrupt: 